<a href="https://colab.research.google.com/github/w0726/ceshi/blob/master/3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls -l 'drive/My Drive/mycode/dataset'

total 2417802
-rw------- 1 root root 2475828275 Aug  5 15:23 10classes.zip


In [3]:
# !unzip 'drive/My Drive/mycode/dataset/10classes.zip' -d 'drive/My Drive/mycode/dataset'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000075.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000076.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000077.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000078.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000079.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000080.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000081.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000082.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000083.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/v_BenchPress_g19_c04/frame000084.jpg  
  inflating: drive/My Drive/mycode/dataset/10classes/

In [5]:
!ls -l 'drive/My Drive/mycode/dataset/10classes'

total 5496
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g01_c01
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g01_c02
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g01_c03
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g01_c04
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g01_c05
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g01_c06
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g02_c01
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g02_c02
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g02_c03
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g02_c04
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g03_c01
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g03_c02
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g03_c03
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g03_c04
drwx------ 2 root root 4096 Aug  5 22:35 v_ApplyEyeMakeup_g03_c05

In [11]:
import os
print(len(os.listdir('drive/My Drive/mycode/dataset/10classes')))
!pwd

1374
/content


In [12]:
import os
import tqdm
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle

In [13]:

# ------------------- label conversion tools ------------------ #


def labels2cat(label_encoder, list):
    return label_encoder.transform(list)


def labels2onehot(onehotEncoder, label_encoder, list):
    return onehotEncoder.transform(label_encoder.transform(list).reshape(-1, 1)).toarray()


def onehot2labels(label_encoder, y_onehot):
    return label_encoder.inverse_transform(np.where(y_onehot == 1)[1]).tolist()


def cat2labels(label_encoder, y_cat):
    return label_encoder.inverse_transform(y_cat).tolist()


def cat2onehot(action_names: list):
    # convert labels -> category
    le = LabelEncoder()
    le.fit(action_names)

    # # show how many classes there are
    # list(le.classes_)

    # convert category -> 1-hot
    action_category = le.transform(action_names).reshape(-1, 1)
    enc = OneHotEncoder()
    enc.fit(action_category)


def getnames(data_path, action_name_path):
    # load UCF101 actions names
    with open(action_name_path, 'rb') as f:
        action_names = pickle.load(f)   # load UCF101 actions names

    # convert labels -> category
    le = LabelEncoder()
    le.fit(action_names)

    # # show how many classes there are
    # list(le.classes_)

    # convert category -> 1-hot
    action_category = le.transform(action_names).reshape(-1, 1)     # type: numpy.ndarray
    enc = OneHotEncoder()
    enc.fit(action_category)

    # # example
    # y = ['HorseRace', 'YoYo', 'WalkingWithDog']
    # y_onehot = labels2onehot(enc, le, y)
    # y2 = onehot2labels(le, y_onehot)

    actions = []
    fnames = os.listdir(data_path)

    all_names = []
    for f in fnames:
        loc1 = f.find('v_')
        loc2 = f.find('_g')
        actions.append(f[(loc1 + 2): loc2])

        all_names.append(f)

    # list all data files
    all_folders_list = all_names              # all video file names
    all_labels_list = labels2cat(le, actions)    # all video labels, type: int

    return all_folders_list, all_labels_list


In [14]:
# ------------------------ 3D CNN module ---------------------- #


def conv3D_output_size(img_size, padding, kernel_size, stride):
    # compute output shape of conv3D
    outshape = (np.floor((img_size[0] + 2 * padding[0] - (kernel_size[0] - 1) - 1) / stride[0] + 1).astype(int),
                np.floor((img_size[1] + 2 * padding[1] - (kernel_size[1] - 1) - 1) / stride[1] + 1).astype(int),
                np.floor((img_size[2] + 2 * padding[2] - (kernel_size[2] - 1) - 1) / stride[2] + 1).astype(int))
    return outshape


class CNN3D(nn.Module):
    def __init__(self, t_dim=120, img_x=90, img_y=120, drop_p=0.2, fc_hidden1=256, fc_hidden2=128, num_classes=50):
        super(CNN3D, self).__init__()

        # set video dimension
        self.t_dim = t_dim
        self.img_x = img_x
        self.img_y = img_y
        # fully connected layer hidden nodes
        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p
        self.num_classes = num_classes
        self.ch1, self.ch2 = 32, 48
        self.k1, self.k2 = (5, 5, 5), (3, 3, 3)  # 3d kernel size
        self.s1, self.s2 = (2, 2, 2), (2, 2, 2)  # 3d strides
        self.pd1, self.pd2 = (0, 0, 0), (0, 0, 0)  # 3d padding

        # compute conv1 & conv2 output shape
        self.conv1_outshape = conv3D_output_size((self.t_dim, self.img_x, self.img_y), self.pd1, self.k1, self.s1)
        self.conv2_outshape = conv3D_output_size(self.conv1_outshape, self.pd2, self.k2, self.s2)

        self.conv1 = nn.Conv3d(in_channels=1, out_channels=self.ch1, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.bn1 = nn.BatchNorm3d(self.ch1)
        self.conv2 = nn.Conv3d(in_channels=self.ch1, out_channels=self.ch2, kernel_size=self.k2, stride=self.s2,
                               padding=self.pd2)
        self.bn2 = nn.BatchNorm3d(self.ch2)
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout3d(self.drop_p)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(self.ch2 * self.conv2_outshape[0] * self.conv2_outshape[1] * self.conv2_outshape[2],
                             self.fc_hidden1)  # fully connected hidden layer
        self.fc2 = nn.Linear(self.fc_hidden1, self.fc_hidden2)
        self.fc3 = nn.Linear(self.fc_hidden2, self.num_classes)  # fully connected layer, output = multi-classes

    def forward(self, x_3d):
        # Conv 1
        x = self.conv1(x_3d)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        # Conv 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.drop(x)
        # FC 1 and 2
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc3(x)

        return x



In [15]:
# ---------------------- Dataloaders ---------------------- #
# for 3DCNN
class Dataset_3DCNN(data.Dataset):
    """Characterizes a dataset for PyTorch"""

    '''
        data_path: the root dir of vedios
        folders: all vedio file names in the dataset, a folder contains a vedio devided by frame
        labels: all labels of the vedio in the dataset
        frames: numpy.ndarray, contains the indexes of frames to be trained in each vedio
        transform: torchvision.transforms
    '''

    def __init__(self, data_path, folders, labels, frames, transform=None):
        "Initialization"
        self.data_path = data_path
        self.labels = labels
        self.folders = folders
        self.transform = transform
        self.frames = frames

    def __len__(self):
        "Denotes the total number of samples"
        return len(self.folders)

    def read_images(self, data_path, folder, use_transform):
        """
            path: the root dir of selected_folder, the same as self.data_path
        """
        X = []
        for i in self.frames:
            image = Image.open(os.path.join(data_path, folder, 'frame{:06d}.jpg'.format(i))).convert('L')

            if use_transform is not None:
                image = use_transform(image)

            X.append(image.squeeze_(0))  # 灰度图像， 1通道，dim=0的元素个数为1时，删去该维度
        X = torch.stack(X, dim=0)   # 增加新维度，在0维上堆叠

        return X

    def __getitem__(self, idx):
        "Generates one sample of data"
        # Select sample
        folder = self.folders[idx]

        # Load data
        data = self.read_images(self.data_path, folder, self.transform).unsqueeze_(0)  # (input) spatial images
        targets = torch.LongTensor([self.labels[idx]])                             # (target) LongTensor are for int64 instead of FloatTensor

        # print(data.shape)
        return data, targets



In [16]:
# ---------------------- Train for 3DCNN ---------------------- #
def train(log_interval, model, device, train_loader, optimizer, epoch):
    start = time.time()
    print(f'{time.asctime(time.localtime(start))}}')
    # set model as training mode
    model.train()

    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device).view(-1, )
        # print(X.size(), y)

        N_count += X.size(0)

        optimizer.zero_grad()
        output = model(X)  # output size = (batch, number of classes)

        loss = F.cross_entropy(output, y)
        losses.append(loss.item())

        # to compute accuracy
        y_pred = torch.max(output, 1)[1]  # y_pred != output
        # print(f'y_pred:\n{y_pred}\ny:\n{y}')
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()

        # show information
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))
    print(f'[{time.time()-start}s]\tTrain Epoch {epoch+1} is done.')
    return losses, scores


# ---------------------- Validation for 3DCNN ---------------------- #
def validation(model, device, optimizer, test_loader, epoch, save_model_path, train_loss, train_score):
    print(f'{time.ctime()}')
    # set model as testing mode
    model.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device).view(-1, )

            output = model(X)

            loss = F.cross_entropy(output, y, reduction='sum')
            test_loss += loss.item()                 # sum up batch loss
            y_pred = output.max(1, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability

            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(test_loader.dataset)

    # to compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100 * test_score))

    # save Pytorch models of best record
    if test_loss < train_loss or test_score > train_score:
      torch.save(model.state_dict(), os.path.join(save_model_path, '3dcnn_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
      torch.save(optimizer.state_dict(), os.path.join(save_model_path, '3dcnn_optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
      print(f"Epoch {epoch+1} model saved!\n")

    return test_loss, test_score



In [17]:
# -------------------- (reload) model prediction ---------------------- #
def Conv3d_final_prediction(model, device, loader):
    model.eval()

    all_y_pred = []
    with torch.no_grad():
        for batch_idx, (X, y) in enumerate(tqdm(loader)):
            # distribute data to device
            X = X.to(device)
            output = model(X)
            y_pred = output.max(1, keepdim=True)[1]  # location of max log-probability as prediction
            all_y_pred.extend(y_pred.cpu().data.squeeze().numpy().tolist())

    return all_y_pred


# -------------------- DRAW PLOT ---------------------- #

def draw(epoch_size: int, epoch_train_losses: np.ndarray, epoch_train_scores: np.ndarray, epoch_test_losses: np.ndarray, epoch_test_scores: np.ndarray):
    # plot
    fig = plt.figure(figsize=(10, 4))
    plt.subplot(121)
    plt.plot(np.arange(1, epoch_size + 1), epoch_train_losses[:, -1])  # train loss (on epoch end)
    plt.plot(np.arange(1, epoch_size + 1), epoch_test_losses)  # test loss (on epoch end)
    plt.title("model loss")
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend(['train', 'test'], loc="upper left")
    # 2nd figure
    plt.subplot(122)
    plt.plot(np.arange(1, epoch_size + 1), epoch_train_scores[:, -1])  # train accuracy (on epoch end)
    plt.plot(np.arange(1, epoch_size + 1), epoch_test_scores)  # test accuracy (on epoch end)
    # plt.plot(histories.losses_val)
    plt.title("training scores")
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.legend(['train', 'test'], loc="upper left")
    title = "./fig_UCF101_3DCNN.png"
    plt.savefig(title, dpi=600)
    plt.close(fig)
    # plt.show()



In [18]:
def main():

    # set path
    data_path = "drive/My Drive/mycode/dataset/10classes"    # define UCF-101 spatial data path
    action_name_path = "drive/My Drive/mycode/classes.pkl"  # load preprocessed action names
    save_model_path = "drive/My Drive/mycode/Conv3D_ckpt/"  # save Pytorch models

    # Detect devices
    use_cuda = torch.cuda.is_available()                   # check if GPU exists
    device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

    # load UCF101 actions names
    all_folders_list, all_labels_list = getnames(data_path, action_name_path)

    # train, test split
    train_list, test_list, train_label, test_label = train_test_split(all_folders_list, all_labels_list, test_size=0.25, random_state=42)

    # image transformation
    img_x, img_y = 256, 342  # resize video 2d frame size
    transform = transforms.Compose([transforms.Resize([img_x, img_y]),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5], std=[0.5])])

    # Get datasat and dataloader
    begin_frame, end_frame, skip_frame = 1, 29, 1   # Select which frame to begin & end in videos
    selected_frames = np.arange(begin_frame, end_frame, skip_frame).tolist()
    train_set = Dataset_3DCNN(data_path, train_list, train_label, selected_frames, transform=transform)
    valid_set = Dataset_3DCNN(data_path, test_list, test_label, selected_frames, transform=transform)

    batch_size = 50
    params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {'batch_size': batch_size, 'shuffle': True}
    train_loader = data.DataLoader(train_set, **params)
    valid_loader = data.DataLoader(valid_set, **params)

    # create model
    # 3D CNN parameters
    num_classes = 10            # number of target category
    fc_hidden1, fc_hidden2 = 256, 256
    dropout = 0.0        # dropout probability

    cnn3d = CNN3D(t_dim=len(selected_frames),   # 时间维度
                  img_x=img_x,
                  img_y=img_y,
                  drop_p=dropout,
                  fc_hidden1=fc_hidden1,
                  fc_hidden2=fc_hidden2,
                  num_classes=num_classes).to(device)

    # Parallelize model to multiple GPUs
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        cnn3d = nn.DataParallel(cnn3d)

    # 训练及验证
    # training parameters
    epoch_size = 10
    learning_rate = 1e-4
    log_interval = 2

    optimizer = torch.optim.Adam(cnn3d.parameters(), lr=learning_rate)   # optimize all cnn parameters

    # record training process
    epoch_train_losses = []
    epoch_train_scores = []
    epoch_test_losses = []
    epoch_test_scores = []

    print(f'Epoch size: {epoch_size}\tBatch size: {batch_size}\tLearning rate: {learning_rate}')
    for epoch in range(epoch_size):
        # train, test model
        train_losses, train_scores = train(log_interval, cnn3d, device, train_loader, optimizer, epoch)
        epoch_test_loss, epoch_test_score = validation(cnn3d, device, optimizer, valid_loader, epoch, save_model_path, train_losses, train_scores)

        # save results
        epoch_train_losses.append(train_losses)
        epoch_train_scores.append(train_scores)
        epoch_test_losses.append(epoch_test_loss)
        epoch_test_scores.append(epoch_test_score)

        # save all train test results
        A = np.array(epoch_train_losses)
        B = np.array(epoch_train_scores)
        C = np.array(epoch_test_losses)
        D = np.array(epoch_test_scores)
        np.save('./3DCNN_epoch_training_losses.npy', A)
        np.save('./3DCNN_epoch_training_scores.npy', B)
        np.save('./3DCNN_epoch_test_loss.npy', C)
        np.save('./3DCNN_epoch_test_score.npy', D)

    # 作图
    draw(epoch_size, A, B, C, D)



In [19]:
if __name__ == "__main__":
  main()

Epoch size: 10	Batch size: 5	Learning rate: 0.0001
Thu Aug  6 13:48:15 2020
Train Epoch: 1 [50/1030 (5%)]	Loss: 12.163386, Accu: 0.00%
Train Epoch: 1 [100/1030 (10%)]	Loss: 2.950215, Accu: 20.00%
Train Epoch: 1 [150/1030 (15%)]	Loss: 2.574661, Accu: 40.00%
Train Epoch: 1 [200/1030 (19%)]	Loss: 3.102270, Accu: 20.00%
Train Epoch: 1 [250/1030 (24%)]	Loss: 1.701903, Accu: 60.00%
Train Epoch: 1 [300/1030 (29%)]	Loss: 0.614473, Accu: 80.00%
Train Epoch: 1 [350/1030 (34%)]	Loss: 0.451758, Accu: 80.00%
Train Epoch: 1 [400/1030 (39%)]	Loss: 2.058846, Accu: 60.00%
Train Epoch: 1 [450/1030 (44%)]	Loss: 2.214818, Accu: 40.00%
Train Epoch: 1 [500/1030 (49%)]	Loss: 1.291678, Accu: 60.00%
Train Epoch: 1 [550/1030 (53%)]	Loss: 0.987819, Accu: 60.00%
Train Epoch: 1 [600/1030 (58%)]	Loss: 0.342510, Accu: 100.00%
Train Epoch: 1 [650/1030 (63%)]	Loss: 0.277398, Accu: 100.00%
Train Epoch: 1 [700/1030 (68%)]	Loss: 1.977032, Accu: 60.00%
Train Epoch: 1 [750/1030 (73%)]	Loss: 0.105377, Accu: 100.00%
Train Ep

RuntimeError: ignored